**<H1>Use watsonx Chat API, and mistralai/mistral-large to make a full conversation using tool calls</H1>**

In [1]:
!pip install requests | tail -n 1
!pip install ibm-cloud-sdk-core | tail -n 1

In [2]:
import getpass, os, requests
from ibm_cloud_sdk_core import IAMTokenManager

In [75]:
api_key = "your_key"
project_id = "your_project_id"
url = "https://us-south.ml.cloud.ibm.com"

In [76]:
class ModelInferanceChat:
    def __init__(self, access_token, project_id):
        self.access_token = access_token
        self.project_id = project_id

    def chat(
        self, 
        model_id, 
        messages, 
        tools = None, 
        tool_choice = None, 
        tool_choice_option = None,
        **kwargs
    ):
        wml_url = f"{url}/ml/v1/text/chat?version=2024-10-07"
        Headers = {
            "Authorization": "Bearer " + self.access_token,
            "Content-Type": "application/json",
            "Accept": "application/json"
        }
        data = {
            "model_id": model_id,
            "messages": messages,
            "tools": tools,
            "tool_choice": tool_choice,
            "tool_choice_option": tool_choice_option,
            "project_id": self.project_id,
        }
        data = data | kwargs
        response = requests.post(wml_url, json=data, headers=Headers)
        if response.status_code == 200:
            return response.json()
        else:
            return response.text

In [77]:
access_token = IAMTokenManager(
    apikey = api_key,
    url = "https://iam.cloud.ibm.com/identity/token"
).get_token()

In [78]:
model = ModelInferanceChat(access_token, project_id)
model_id = "mistralai/mistral-large"

In [79]:
messages = [
    {"role": "system", "content": "You are a helpful customer support assistant. Use the supplied tools to assist the user. Be short."},
    {'role': 'user', 'content': 'What is the delivery date of my order?'}, 
]

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_delivery_date",
            "description": "Get the delivery date for a customers order. Call this whenever you need to know the delivery date, for example when a customer asks: Where is my package",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": {
                        "type": "string",
                        "description": "ID of the order",
                    }
                },
                "required": ["order_id"],
            },
        },
    }
]


tool_choice_option = "auto"
tool_response = model.chat(
    model_id=model_id,
    messages=messages, 
    tools=tools, 
    tool_choice_option=tool_choice_option
)

In [80]:
import json

response = json.dumps(tool_response["choices"][0]["message"], indent=4)
print(response)

{
    "role": "assistant",
    "content": " Please provide your order ID."
}


In [81]:
messages.append(tool_response["choices"][0]["message"])
print(json.dumps(messages, indent=4))

[
    {
        "role": "system",
        "content": "You are a helpful customer support assistant. Use the supplied tools to assist the user. Be short."
    },
    {
        "role": "user",
        "content": "What is the delivery date of my order?"
    },
    {
        "role": "assistant",
        "content": " Please provide your order ID."
    }
]


In [82]:
messages.append({"role": "user", "content": "sure! it is order 87678"})
print(json.dumps(messages, indent=4))

[
    {
        "role": "system",
        "content": "You are a helpful customer support assistant. Use the supplied tools to assist the user. Be short."
    },
    {
        "role": "user",
        "content": "What is the delivery date of my order?"
    },
    {
        "role": "assistant",
        "content": " Please provide your order ID."
    },
    {
        "role": "user",
        "content": "sure! it is order 87678"
    }
]


In [83]:
tool_response = model.chat(
    model_id=model_id,
    messages=messages, 
    tools=tools, 
    tool_choice_option=tool_choice_option
)

In [84]:
response = json.dumps(tool_response["choices"][0]["message"], indent=4)
print(response)

{
    "role": "assistant",
    "tool_calls": [
        {
            "id": "chatcmpl-tool-075a24c271f1460dbe542e57175b84d5",
            "type": "function",
            "function": {
                "name": "get_delivery_date",
                "arguments": "{\"order_id\": \"87678\"}"
            }
        }
    ]
}


In [86]:
from datetime import datetime
# This is the function that we want the model to be able to call
def get_delivery_date(order_id: str) -> datetime:
    # function logic (call external service, API, DB...)
    return datetime.now()

In [87]:
# Extract the arguments for get_delivery_date
tool_call = tool_response["choices"][0]["message"]["tool_calls"][0]
arguments = json.loads(tool_call['function']['arguments'])

order_id = arguments.get('order_id')

# Call the get_delivery_date function with the extracted order_id
delivery_date = get_delivery_date(order_id)

In [88]:
# Create a message containing the result of the function call
function_call_result_message = {
    "role": "tool",
    "content": json.dumps({
        "order_id": order_id,
        "delivery_date": delivery_date.strftime('%Y-%m-%d')
    }),
    "tool_call_id": tool_response['choices'][0]['message']['tool_calls'][0]['id']
}

In [89]:
messages.append(function_call_result_message)
print(json.dumps(messages, indent=4))

[
    {
        "role": "system",
        "content": "You are a helpful customer support assistant. Use the supplied tools to assist the user. Be short."
    },
    {
        "role": "user",
        "content": "What is the delivery date of my order?"
    },
    {
        "role": "assistant",
        "content": " Please provide your order ID."
    },
    {
        "role": "user",
        "content": "sure! it is order 87678"
    },
    {
        "role": "tool",
        "content": "{\"order_id\": \"87678\", \"delivery_date\": \"2024-10-14\"}",
        "tool_call_id": "chatcmpl-tool-075a24c271f1460dbe542e57175b84d5"
    }
]


In [90]:
tool_response = model.chat(
    model_id=model_id,
    messages=messages, 
    tools=tools, 
    tool_choice_option=tool_choice_option
)

In [91]:
response = json.dumps(tool_response["choices"][0]["message"], indent=4)
print(response)

{
    "role": "assistant",
    "content": " Your order has been scheduled for delivery on October 14, 2024."
}
